<a href="https://colab.research.google.com/github/whyamistudyingcs/FYP_attack/blob/main/Hugging_face_model_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The TextAttack ecosystem: search, transformations, and constraints

#installation


Please remember to run **pip3 install textattack[tensorflow]** in your notebook enviroment before the following codes:

In [ ]:
!pip3 install textattack[tensorflow]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 58.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64

In [ ]:
!pip3 install tensorflow==2.14
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboar

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


# Evaluation

In [ ]:
import transformers
from datasets import load_dataset, load_dataset_builder, DatasetDict
import evaluate

In [ ]:
ds_builder = load_dataset_builder("imdb")
ds_builder.info.description

'Large Movie Review Dataset.\nThis is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.'

In [ ]:
ds_builder.info.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [ ]:
imdb_dataset = load_dataset("imdb", split="train")

def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }

small_imdb_dataset = DatasetDict(
    val = imdb_dataset.shuffle(seed=1111).select(range(128)).map(truncate)
)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [ ]:
small_imdb_dataset['val'][:10]

{'text': ["Probably Jackie Chan's best film in the 1980s, and the one that put him on the map. The scale of this self-directed police drama is evident from the opening and closing scenes, during which a squatters' village and shopping mall are demolished. There are, clearly, differences between the original Chinese",
  'A wonderful movie! Anyone growing up in an Italian family will definitely see themselves in these characters. A good family movie with sadness, humor, and very good acting from all. You will enjoy this movie!! We need more like it.',
  'HORRENDOUS! Avoid like the plague. I would rate this in the top 10 worst movies ever. Special effects, acting, mood, sound, etc. appear to be done by day care students...wait, I have seen programs better than this. Opens like a soft porn show with a blurred nude female doing a',
  'And I absolutely adore Isabelle Blais!!! She was so cute in this movie, and far different from her role in "Quebec-Montreal" where she was more like a man-eat

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import torch


tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news").to("cuda")

small_tokenized_dataset = small_imdb_dataset.map(
    lambda example: tokenizer(example['text'], padding=True, truncation=True),
    batched=True,
    batch_size=16
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["text"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

In [ ]:
input_str = "I love captain America"

model_inputs = tokenizer(input_str, return_tensors="pt").to("cuda")
print(model_inputs)
model_outputs= model(input_ids=model_inputs.input_ids, attention_mask=model_inputs.attention_mask)
print(model_outputs)

{'input_ids': tensor([[  35,  564, 3064,  640,    4,    3]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}
XLNetForSequenceClassificationOutput(loss=None, logits=tensor([[-1.2182,  1.0525]], device='cuda:0', grad_fn=<AddmmBackward0>), mems=(tensor([[[ 0.0249,  0.0009, -0.0455,  ..., -0.0290,  0.0105, -0.0097]],

        [[-0.0832, -0.0247,  0.0919,  ..., -0.0688,  0.0308, -0.0351]],

        [[ 0.0035, -0.0389,  0.0663,  ...,  0.1375, -0.0208,  0.1208]],

        [[ 0.0555, -0.0034,  0.0116,  ..., -0.0812, -0.0463,  0.0134]],

        [[ 0.0783, -0.0579, -0.0900,  ...,  0.0500,  0.0627, -0.0526]],

        [[ 0.0182, -0.0010, -0.1494,  ...,  0.0022, -0.0015,  0.0189]]],
       device='cuda:0'), tensor([[[-0.3275,  0.1407, -1.4588,  ..., -0.5534, -0.0975,  0.9138]],

        [[-2.2595, -0.6832,  0.8524,  ..., -0.8340,  0.7881,  0.0224]],

        [[-0.5870, -1.4000,  0.1115,  ...,  1.69

In [ ]:
metric = evaluate.load("accuracy")
model.eval()

for batch in eval_dataloader:
  batch = {k: v.to("cuda") for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8984375}

# attack

In [ ]:
# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/distilbert-base-uncased-ag-news")
model = AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-uncased-ag-news").to("cuda")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)


# Import the dataset
from textattack.datasets import HuggingFaceDataset

dataset = HuggingFaceDataset("ag_news", None, "test")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

textattack: Loading datasets dataset ag_news, split test.


### Creating the attack
Let's keep it simple: let's use a greedy search method, and let's not use any constraints for now.

In [ ]:
!pip install sentence_transformers
!pip install torchfile

  Preparing metadata (setup.py) ... done
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5694 sha256=40021d0633e7940d89b98a143d7795d670938b2239896193e77270bb8566b7c0
  Stored in directory: /root/.cache/pip/wheels/c7/e9/87/1c51daf8e468d5c14931f8ac3344880f903ba96b063675cac2
Successfully built torchfile


In [ ]:
from textattack.attack_recipes import PWWSRen2019, A2TYoo2021, BERTAttackLi2020, FasterGeneticAlgorithmJia2019
from textattack import Attacker

attack = BERTAttackLi2020.build(model_wrapper)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
# customized attack
from textattack.goal_functions import UntargetedClassification
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapMaskedLM
from textattack import Attack

transformation = WordSwapMaskedLM(method="bert-attack", max_candidates=24)
constraints = [RepeatModification(), StopwordModification()]
constraints.append(MaxWordsPerturbed(max_percent=0.3))

use_constraint = UniversalSentenceEncoder(
    threshold=0.2,
    metric="cosine",
    compare_against_original=True,
    window_size=None,
)

constraints.append(use_constraint)
goal_function = UntargetedClassification(model_wrapper)
search_method = GreedyWordSwapWIR(wir_method="unk")

attack = Attack(goal_function, constraints, transformation, search_method)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Let's print our attack to see all the parameters:

In [ ]:
print(attack)


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  24
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.3
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
)


In [ ]:
print(dataset)

### Using the attack

Let's use our attack to successfully attack 10 samples.

In [ ]:
from tqdm import tqdm  # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger  # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=32)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  24
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.3
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 





  0%|          | 0/32 [00:00<?, ?it/s]

  3%|▎         | 1/32 [00:09<04:47,  9.27s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   3%|▎         | 1/32 [00:09<04:48,  9.29s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (98%)]] --> [[World (55%)]]

Fears for T N pension after talks [[Unions]] [[representing]] workers at Turner   [[Newall]] say they are 'disappointed' after talks with stricken [[parent]] [[firm]] Federal Mogul.

Fears for T N pension after talks [[unionist]] [[united]] workers at Turner   [[newnew]] say they are 'disappointed' after talks with stricken [[party]] [[un]] Federal Mogul.






[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   6%|▋         | 2/32 [03:05<46:20, 92.69s/it]

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   6%|▋         | 2/32 [03:05<46:20, 92.69s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.




...

--------------------------------------------- Result 4 ---------------------------------------------
[[Sci/tech (99%)]] --> [[Sports (52%)]]

[[Prediction]] Unit Helps [[Forecast]] Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful [[maps]], figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.

[[d]] Unit Helps [[scout]] Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful [[map]], figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.






[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:  16%|█▌        | 5/32 [03:20<18:05, 40.19s/it]

[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:  16%|█▌        | 5/32 [03:20<18:05, 40.20s/it]

--------------------------------------------- Result 5 ---------------------------------------------
[[Sci/tech (99%)]] --> [[World (52%)]]

[[Calif]]. Aims to Limit Farm-Related Smog ([[AP]]) [[AP]] - Southern California's smog-fighting agency went after emissions of the [[bovine]] [[variety]] Friday, adopting the nation's first rules to reduce [[air]] [[pollution]] from [[dairy]] cow manure.

.[[rec]]. Aims to Limit Farm-Related Smog ([[abs]]) [[pac]] - Southern California's smog-fighting agency went after emissions of the [[bofe]] [[market]] Friday, adopting the nation's first rules to reduce [[high]] [[income]] from [[milk]] cow manure.






[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:  19%|█▉        | 6/32 [09:36<41:40, 96.16s/it]

[Succeeded / Failed / Skipped / Total] 3 / 3 / 0 / 6:  19%|█▉        | 6/32 [09:36<41:40, 96.16s/it]

--------------------------------------------- Result 6 ---------------------------------------------
[[Sci/tech (99%)]] --> [[[FAILED]]]

Open Letter Against British Copyright Indoctrination in Schools The British Department for Education and Skills (DfES) recently launched a "Music Manifesto" campaign, with the ostensible intention of educating the next generation of British musicians. Unfortunately, they also teamed up with the music industry (EMI, and various artists) to make this popular. EMI has apparently negotiated their end well, so that children in our schools will now be indoctrinated about the illegality of downloading music.The ignorance and audacity of this got to me a little, so I wrote an open letter to the DfES about it. Unfortunately, it's pedantic, as I suppose you have to be when writing to goverment representatives. But I hope you find it useful, and perhaps feel inspired to do something similar, if or when the same thing has happened in your area.




### Visualizing attack results

We are logging `AttackResult` objects using a `CSVLogger`. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set `color_method` to `'html'`, the attack results will display their differences, in color, in HTML. Using `IPython` utilities and `pandas`

In [ ]:
import pandas as pd

pd.options.display.max_colwidth = (
    480  # increase colum width so we can actually read the examples
)

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


NameError: ignored

### Conclusion
We can examine these examples for a good idea of how many words had to be changed to "banana" to change the prediction score from the correct class to another class. The examples without perturbed words were originally misclassified, so they were skipped by the attack. Looks like some examples needed only a couple "banana"s, while others needed up to 17 "banana" substitutions to change the class score. Wow! 🍌

### Appendices: Attacking Custom Samples

We can also attack custom data samples, like these ones I just made up!

In [ ]:


custom_dataset = [
    ("And while Fushimi gives a kind speech at the end, her unyielding response to Saori’s desperate questions also doesn’t look much better in hindsight after we’ve gotten to know her better.", 1)
]

attack_args = AttackArgs(num_examples=4)

dataset = Dataset(custom_dataset)

attacker = Attacker(attack, dataset, attack_args)

results_iterable = attacker.attack_dataset()

logger = CSVLogger(color_method="html")

for result in results_iterable:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML

display(HTML(logger.df[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Attempting to attack 4 samples when only 1 are available.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 





  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:05<00:17,  5.99s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  25%|██▌       | 1/4 [00:06<00:18,  6.06s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (51%)]] --> [[0 (60%)]]

And while Fushimi gives a kind speech at the end, her unyielding response to Saori’s desperate questions also doesn’t look much better in hindsight after we’ve gotten to [[know]] her better.

And while Fushimi gives a kind speech at the end, her unyielding response to Saori’s desperate questions also doesn’t look much better in hindsight after we’ve gotten to [[fuck]] her better.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 2.94%  |
| Average num. words per input: | 34.0   |
| Avg num q


textattack: Logging to CSV at path results.csv
textattack: CSVLogger exiting without calling flush().


,original_text,perturbed_text
0,"And while Fushimi gives a kind speech at the end, her unyielding response to Saori’s desperate questions also doesn’t look much better in hindsight after we’ve gotten to know her better.","And while Fushimi gives a kind speech at the end, her unyielding response to Saori’s desperate questions also doesn’t look much better in hindsight after we’ve gotten to fuck her better."
